use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [4]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle

/root/autodl-tmp/xiaolong/.conda/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [6]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [7]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 2048
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id]
            product_emb = product_embeddings[batch_product_id]
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [8]:
def normalize_scores(score_df, score_name, normalized_score_name):
    score_df_g = cudf.from_pandas(score_df)
    score_df_g['exp_score'] = np.exp(score_df_g[score_name].to_numpy())
    scores_sum_g = score_df_g[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum_g.reset_index(inplace=True)
    scores_sum_g = scores_sum_g.sort_values(by=['sess_id'], ascending=True)
    scores_sum_g.reset_index(drop=True, inplace=True)
    scores_sum_g.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df_g = score_df_g.merge(scores_sum_g, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df_g = merged_score_df_g.sort_values(by=['sess_id', 'product'])
    merged_score_df_g.reset_index(drop=True, inplace=True)
    
    merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    del scores_sum_g
    del merged_score_df_g 

# Merge valid score

In [7]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_no_hist_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [8]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [9]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_DE/product_embeddings_2023-05-04-10-40-13.pt'
DE_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_DE/valid_embeddings_2023-05-04-10-40-21.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_JP/product_embeddings_2023-05-04-10-40-59.pt'
JP_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_JP/valid_embeddings_2023-05-04-10-41-04.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_UK/product_embeddings_2023-05-04-10-46-27.pt'
UK_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_UK/valid_embeddings_2023-05-04-10-46-34.pt'

In [10]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/c76eddf0a07106ffcce7ce8010856a3b'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/81a71d0a18766af84b3beab69bf53e69'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/250dbc09c30162452e00486051e47756'

In [11]:
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [12]:
merged_candidates = read_merged_candidates_feature()
valid_sessions = read_valid_sessions()
EMBED_DIM = 128
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [13]:
# sess embeddings 
valid_DE_query_emb = torch.load(DE_valid_embeddings_path, map_location='cpu')
valid_JP_query_emb = torch.load(JP_valid_embeddings_path, map_location='cpu')
valid_UK_query_emb = torch.load(UK_valid_embeddings_path, map_location='cpu')
valid_query_embeddings = torch.empty(len(valid_sessions), EMBED_DIM)
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'DE'].index).tolist()] = valid_DE_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'JP'].index).tolist()] = valid_JP_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'UK'].index).tolist()] = valid_UK_query_emb

In [15]:
# product_embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [16]:
merged_candidates_sasrec = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [17]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [18]:
merged_candidates_sasrec_g = cudf.from_pandas(merged_candidates_sasrec)
product_id_df_g = cudf.from_pandas(product_id_df)

In [19]:
merged_candidates_sasrec_score_g = merged_candidates_sasrec_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_sasrec_score_g['dataset_id'] = merged_candidates_sasrec_score_g['dataset_id'].fillna(0)
merged_candidates_sasrec_score_g.drop(columns=['locale'], inplace=True)
merged_candidates_sasrec_score_g = merged_candidates_sasrec_score_g.sort_values(by=['sess_id', 'product'])
merged_candidates_sasrec_score_g.reset_index(drop=True, inplace=True)
merged_candidates_sasrec_score = merged_candidates_sasrec_score_g.to_pandas()

In [20]:
del merged_candidates_sasrec_g
del product_id_df_g
del merged_candidates_sasrec_score_g

In [21]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] = \
        merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_495341/2146751149.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] = \


In [22]:
valid_query_embeddings = valid_query_embeddings.to('cuda:0')
product_embeddings = product_embeddings.to('cuda:0')

In [23]:
merged_candidates_sasrec_score['sasrec_scores_2'] = get_scores(merged_candidates_sasrec_score, valid_query_embeddings, product_embeddings)

100%|██████████| 41215/41215 [01:34<00:00, 436.21it/s] 


In [24]:
normalize_scores(merged_candidates_sasrec_score, 'sasrec_scores_2', 'sasrec_normalized_scores_2')

In [25]:
merged_candidates['sasrec_scores_2'] = merged_candidates_sasrec_score['sasrec_scores_2']
merged_candidates['sasrec_normalized_scores_2'] = merged_candidates_sasrec_score['sasrec_normalized_scores_2']

In [26]:
cast_dtype(merged_candidates)
merged_candidates.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [25]:
merged_candidates_sasrec_score

,sess_id,sess_locale,product,dataset_id,sasrec_scores_2,sasrec_normalized_scores_2,exp_score,score_sum
0,0,UK,B000OPPVCS,950401,11.972421,0.000199,158327.458974,793967796.290662
1,0,UK,B000V599Y2,1086285,13.152878,0.000649,515492.383636,793967796.290662
2,0,UK,B0018HH444,1296855,5.606023,0.0,272.060185,793967796.290662
3,0,UK,B0079JI4DU,913338,0.000000,0.0,1.0,793967796.290662
4,0,UK,B0079JI4EY,913338,0.000000,0.0,1.0,793967796.290662
...,...,...,...,...,...,...,...,...
84209438,361580,DE,B0B6QWY8Q7,0,0.000000,0.0,1.0,887985790.377148
84209439,361580,DE,B0BB7YSRBX,52408,9.163816,0.000011,9545.417277,887985790.377148
84209440,361580,DE,B0BB7ZMGY8,20389,11.256460,0.000087,77378.186355,887985790.377148
84209441,361580,DE,B0BD4CP7N3,491692,-3.778687,0.0,0.022853,887985790.377148


In [14]:
merged_candidates.query("sess_id == 2").query("product=='B07VYSSRL7'")

,sess_id,sess_locale,product,target


In [28]:
# verify sasrec scores
merged_candidates[merged_candidates['sess_id'] == 2].sort_values(by=['sasrec_scores_2'], ascending=False)[['sess_id', 'sess_locale', 'product', 'sasrec_scores_2', 'sasrec_normalized_scores_2']].iloc[:20]

,sess_id,sess_locale,product,sasrec_scores_2,sasrec_normalized_scores_2
443,2,UK,B00L529BAC,19.369759,0.522441
617,2,UK,B096Y8RWJN,18.264015,0.172909
655,2,UK,B0B1VNZW6H,17.547401,0.084450
645,2,UK,B09V5QFBLP,16.632498,0.033827
538,2,UK,B07CFSC84Q,16.412687,0.027152
477,2,UK,B01EYGW86Y,16.076199,0.019394
516,2,UK,B01MDOBUCC,15.382807,0.009695
481,2,UK,B01EYGY1N2,15.159929,0.007758
598,2,UK,B08M1DXCLX,15.106356,0.007353
648,2,UK,B09XBS6WCX,15.081671,0.007174


In [27]:
valid_sessions

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP
...,...,...,...
361576,['B08HH6L4PB' 'B08L8N8HDR'],B00TS5UXGY,UK
361577,['B08X4L1KLZ' 'B09BBX1T4S' 'B09D76FT9D'],B09BCM5NL1,JP
361578,['B0098G6L3M' 'B00ELRLP3O' 'B00PLXGK82' 'B09GS...,B0BC38GHB4,DE
361579,['B07Q2CNLY3' 'B07Q2CNLY3' 'B07BR7DZWN' 'B07Q2...,B08H8SYLMQ,DE


In [34]:
merged_candidates

,sess_id,sess_locale,product,target,sasrec_scores_2,sasrec_normalized_scores_2
0,0,UK,B000OPPVCS,0.0,11.972421,1.994129e-04
1,0,UK,B000V599Y2,0.0,13.152878,6.492611e-04
2,0,UK,B0018HH444,0.0,5.606023,3.426590e-07
3,0,UK,B0079JI4DU,0.0,0.000000,1.259497e-09
4,0,UK,B0079JI4EY,0.0,0.000000,1.259497e-09
...,...,...,...,...,...,...
84209438,361580,DE,B0B6QWY8Q7,0.0,0.000000,1.126144e-09
84209439,361580,DE,B0BB7YSRBX,0.0,9.163816,1.074952e-05
84209440,361580,DE,B0BB7ZMGY8,0.0,11.256460,8.713899e-05
84209441,361580,DE,B0BD4CP7N3,0.0,-3.778687,2.573541e-11


# Merge test score

In [1]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_test_no_hist_feature.parquet'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [2]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_DE/product_embeddings_2023-05-04-10-40-13.pt'
DE_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_DE/predict_embeddings_2023-05-04-10-40-46.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_JP/product_embeddings_2023-05-04-10-40-59.pt'
JP_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_JP/predict_embeddings_2023-05-04-10-41-27.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_UK/product_embeddings_2023-05-04-10-46-27.pt'
UK_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/query_embeddings/SASRec_Next/kdd_cup_2023_UK/predict_embeddings_2023-05-04-10-46-59.pt'

In [9]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [10]:
merged_candidates = read_merged_candidates_feature_test()
test_sessions = read_test_sessions()
EMBED_DIM = 128
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [11]:
# sess embeddings 
test_DE_query_emb = torch.load(DE_test_embeddings_path, map_location='cpu')
test_JP_query_emb = torch.load(JP_test_embeddings_path, map_location='cpu')
test_UK_query_emb = torch.load(UK_test_embeddings_path, map_location='cpu')
test_query_embeddings = torch.cat(
    [test_DE_query_emb[test_sessions['locale'] == 'DE'], test_JP_query_emb[test_sessions['locale'] == 'JP'], test_UK_query_emb[test_sessions['locale'] == 'UK']],
    dim=0)

In [12]:
test_query_embeddings.shape

torch.Size([316971, 128])

In [13]:
# product embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [14]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/c76eddf0a07106ffcce7ce8010856a3b'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/81a71d0a18766af84b3beab69bf53e69'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/250dbc09c30162452e00486051e47756'
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [15]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [16]:
merged_candidates_sasrec = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [17]:
merged_candidates_sasrec_g = cudf.from_pandas(merged_candidates_sasrec)
product_id_df_g = cudf.from_pandas(product_id_df)

In [18]:
merged_candidates_sasrec_score_g = merged_candidates_sasrec_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_sasrec_score_g['dataset_id'] = merged_candidates_sasrec_score_g['dataset_id'].fillna(0)
merged_candidates_sasrec_score_g.drop(columns=['locale'], inplace=True)
merged_candidates_sasrec_score_g = merged_candidates_sasrec_score_g.sort_values(by=['sess_id', 'product'])
merged_candidates_sasrec_score_g.reset_index(drop=True, inplace=True)
merged_candidates_sasrec_score = merged_candidates_sasrec_score_g.to_pandas()

In [19]:
del merged_candidates_sasrec_score_g
del product_id_df_g
del merged_candidates_sasrec_g

In [20]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] = \
        merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_504565/2146751149.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_sasrec_score['dataset_id'][merged_candidates_sasrec_score['sess_locale'] == locale] = \


In [21]:
merged_candidates_sasrec_score['sasrec_scores_2'] = get_scores(merged_candidates_sasrec_score, test_query_embeddings, product_embeddings)

100%|██████████| 33901/33901 [00:43<00:00, 773.17it/s] 


In [22]:
normalize_scores(merged_candidates_sasrec_score, 'sasrec_scores_2', 'sasrec_normalized_scores_2')

In [23]:
merged_candidates['sasrec_scores_2'] = merged_candidates_sasrec_score['sasrec_scores_2']
merged_candidates['sasrec_normalized_scores_2'] = merged_candidates_sasrec_score['sasrec_normalized_scores_2']

In [56]:
cast_dtype(merged_candidates)
merged_candidates.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [33]:
merged_candidates[(merged_candidates['sess_id'] == 200000)].sort_values(by='sasrec_normalized_scores_2', ascending=False)[['sess_locale', 'product', 'sasrec_normalized_scores_2', 'sasrec_scores_2']][:20]

,sess_locale,product,sasrec_normalized_scores_2,sasrec_scores_2
43632197,JP,B07WVXGFCJ,0.212465,19.104816
43632326,JP,B09Y32HQS5,0.123382,18.561321
43632399,JP,B0BFQGJRBY,0.105723,18.406866
43632296,JP,B09NXS2J43,0.080563,18.135077
43632310,JP,B09TVFSQ7F,0.066885,17.949009
43632377,JP,B0BBH5S35X,0.037597,17.372955
43632176,JP,B07H27J698,0.035764,17.322983
43632335,JP,B0B24Z6MCB,0.033377,17.253918
43632331,JP,B09Z2KGR1Y,0.031402,17.192924
43632193,JP,B07SDHFX36,0.031242,17.187799


In [34]:
test_sessions.loc[200000]['prev_items']

"['B09NBQKRPC' 'B09NBQKRPC' 'B0BHQQQK2D']"